# Proceso a seguir

1. Importar datos de entrenamiento
1. Aumentar para incluir negativos
3. Split train val
4. Preprocesamiento de train
5. Entrenar modelo con datos de train
5. Predicciones con val
6. Evaluación

In [1]:
import pandas as pd
import os

train_file = "../../datasets/customer_purchases/customer_purchases_train.csv"
train_file = os.path.abspath(train_file)
train_df = pd.read_csv(train_file)

train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7289 entries, 0 to 7288
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   purchase_id             7289 non-null   int64  
 1   customer_id             7289 non-null   object 
 2   customer_date_of_birth  7289 non-null   object 
 3   customer_gender         5738 non-null   object 
 4   customer_signup_date    7289 non-null   object 
 5   item_id                 7289 non-null   object 
 6   item_title              7289 non-null   object 
 7   item_category           7289 non-null   object 
 8   item_price              7289 non-null   float64
 9   item_img_filename       7289 non-null   object 
 10  item_avg_rating         7244 non-null   float64
 11  item_num_ratings        7289 non-null   int64  
 12  item_release_date       7289 non-null   object 
 13  purchase_timestamp      7289 non-null   object 
 14  customer_item_views     7289 non-null   

In [2]:
from data_processing import *

X, y = read_train_data()

df saved to D:\ml25_escuelita\src\ml25\P01_customer_purchases\boilerplate\data_processing.py\..\..\..\datasets\customer_purchases\customer_features.csv
df saved to D:\ml25_escuelita\src\ml25\P01_customer_purchases\boilerplate\data_processing.py\..\..\..\datasets\customer_purchases\processed_train.csv


In [5]:
train_df = train_df.drop(columns="label")
train_df.info()
customer_feat = read_csv("customer_features")
merged = pd.merge(train_df, customer_feat, on="customer_id", how="left")

processed = process_df(merged, training=False)
processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7289 entries, 0 to 7288
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   purchase_id             7289 non-null   int64  
 1   customer_id             7289 non-null   object 
 2   customer_date_of_birth  7289 non-null   object 
 3   customer_gender         5738 non-null   object 
 4   customer_signup_date    7289 non-null   object 
 5   item_id                 7289 non-null   object 
 6   item_title              7289 non-null   object 
 7   item_category           7289 non-null   object 
 8   item_price              7289 non-null   float64
 9   item_img_filename       7289 non-null   object 
 10  item_avg_rating         7244 non-null   float64
 11  item_num_ratings        7289 non-null   int64  
 12  item_release_date       7289 non-null   object 
 13  purchase_timestamp      7289 non-null   object 
 14  customer_item_views     7289 non-null   

ValueError: columns are missing: {'label'}

In [ ]:
test_df = read_csv("customer_purchases_test")
test_df.info()
customer_feat = read_csv("customer_features")
#test_df = pd.merge(test_df, customer_feat, on="customer_id")
# agregar features derivados del cliente al dataset
merged = pd.merge(test_df, customer_feat, on="customer_id", how="left")
# Procesamiento de datos
processed = preprocess(merged, training=False)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import make_scorer, roc_auc_score

def tune_models(X, y):
    """
    Tune SVM and XGBoost using GridSearchCV with ROC-AUC as scoring.
    
    Prints best hyperparameters for each model.
    
    Args:
        X: feature DataFrame or array
        y: labels Series or array
    
    Returns:
        best_estimators: dict with 'svm' and 'xgboost' best estimators
        best_scores: dict with best ROC-AUC scores for each
    """
    
    # ---------- SVM ----------
    svm = SVC(probability=True)
    svm_param_grid = {
        "C": [0.1, 1.0, 10],
        "kernel": ["rbf", "poly"],
        "gamma": ["scale", "auto"]
    }
    
    svm_grid = GridSearchCV(
        svm,
        svm_param_grid,
        scoring=make_scorer(roc_auc_score, needs_proba=True),
        cv=5,
        n_jobs=-1,
        verbose=2
    )
    
    print("Tuning SVM...")
    svm_grid.fit(X, y)
    best_svm = svm_grid.best_estimator_
    best_svm_score = svm_grid.best_score_
    print("Best SVM Hyperparameters:", svm_grid.best_params_)
    print("Best SVM ROC-AUC:", best_svm_score)
    
    # ---------- XGBoost ----------
    xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    xgb_param_grid = {
        "n_estimators": [100, 300, 500],
        "max_depth": [3, 6, 9],
        "learning_rate": [0.01, 0.1, 0.2],
        "subsample": [0.7, 1.0],
        "colsample_bytree": [0.7, 1.0]
    }
    
    xgb_grid = GridSearchCV(
        xgb,
        xgb_param_grid,
        scoring=make_scorer(roc_auc_score, needs_proba=True),
        cv=5,
        n_jobs=-1,
        verbose=2
    )
    
    print("Tuning XGBoost...")
    xgb_grid.fit(X, y)
    best_xgb = xgb_grid.best_estimator_
    best_xgb_score = xgb_grid.best_score_
    print("Best XGBoost Hyperparameters:", xgb_grid.best_params_)
    print("Best XGBoost ROC-AUC:", best_xgb_score)
    
    best_estimators = {"svm": best_svm, "xgboost": best_xgb}
    best_scores = {"svm": best_svm_score, "xgboost": best_xgb_score}
    
    return best_estimators, best_scores


In [ ]:
X, y = read_train_data()
best_models, best_scores = tune_models(X, y)

# Access best estimators if needed
svm_model = best_models["svm"]
xgb_model = best_models["xgboost"]